In [1]:
# ##google colab用にchainerのインストール
# !curl https://colab.chainer.org/install | sh -

In [2]:
# ##google driveのマウント
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# ##google driveの場合はinstallする
# !pip install hyperopt

In [4]:
import numpy as np
import chainer

from chainer import cuda,Function,report,training,utils,Variable
from chainer import datasets,iterators,optimizers,serializers
from chainer import Link,Chain,ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.datasets import tuple_dataset
import matplotlib.pyplot as plt
from hyperopt import fmin, tpe, hp
from hyperopt.pyll import scope


%matplotlib inline

In [5]:
##GPU環境の設定を確認
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

GPU availability: True
cuDNN availablility: True


In [6]:
##colab用にデータのpathを指定
##My Driveのカンマはpath指定の時にはいらない

# DATA_PATH = "/content/drive/My Drive/resarch"
# !ls /content/drive/'My Drive'/resarch/

DATA_PATH='./'

In [7]:
##データの読み込み
import pickle

## train data
f=open(DATA_PATH+"/x_train_corr_high.txt","rb")
pre_data_train=pickle.load(f)
f=open(DATA_PATH+"/y_train_corr_high.txt","rb")
label_train=pickle.load(f)

### 不均衡データなので、
#### 1,正解データと不例データから均等にサンプリングして複数データセットをひとつにまとめたものをデータセットにする<br>
#### 2,アンダーサンプリングで、正例データの数に不例データの数を合わせる

In [8]:
##validationデータを正例、不例の一定割合データセットに分ける
##正例、不例のindexを取得しておく

buy=[]
no_buy=[]
for i in range(len(label_train)):
    if label_train[i]==1:
        buy.append(i)
    else:
        no_buy.append(i)
##train時にはバッチサイズ64として、正例と不例を32ずつindexをサンプリングしたデータセットを複数作成して、まとめてデータセットとする
##正例、不例一定割合のデータセット作成
x=[]
y=[]

'''make train data'''
'''1,正解データと不例データから均等にサンプリングして複数データセットをひとつにまとめたものをデータセットにする'''
for i in range(2000):
    posi=np.random.choice(buy,32,replace = False)
    nega=np.random.choice(no_buy,32,replace = False)
    for i in posi:
        x.append(pre_data_train[i])
        y.append(label_train[i])
    for i in nega:
        x.append(pre_data_train[i])
        y.append(label_train[i])

# '''↓2のサンプリングではvalidationのaccuracyがかなり下がるので、使わない'''    
# # '''2,アンダーサンプリングで、正例データの数に不例データの数を合わせる'''
# # nega=np.random.choice(no_buy,len(buy),replace = False)
# # for i in buy:
# #     x.append(pre_data_train[i])
# #     y.append(label_train[i])
# # for i in nega:
# #     x.append(pre_data_train[i])
# #     y.append(label_train[i])
    

In [9]:
# '''特に不均衡として処理しない場合'''
# x=pre_data_train
# y=label_train

In [10]:
sum(y)/len(y)

array(0.5)

**hyperoptで調べたいこと<br>
・convolutionの数、チャネルの数、filaの数、padの数<br>
・最適化関数の設定**<br>
※フィルタサイズとパディングは固定して、いくつか試して良い物を選択する。仮に複数フィルタサイズを試したいなら、for文を積むように書いて行う


In [11]:
#ネットワーク定義

out_chanel1=128
out_chanel2=32
out_chanel3=64
k_size=3
pad_size=1
class Purchase(Chain):
    def __init__(self,activate):
        super(Purchase , self).__init__(
            ##k_size:3
            conv0=L.Convolution2D(1,out_chanel1,ksize=5,pad=1),
            ##
            conv1=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv2=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv3=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            ##
            conv4=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv5=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv6=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            ##
            conv7=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv8=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv9=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            ##
            conv10=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv11=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv12=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            ##
            conv13=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv14=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv15=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            ##
            conv16=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv17=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv18=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            ##
            conv19=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv20=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv21=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            ##
            conv22=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv23=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv24=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            ##
            conv25=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv26=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            conv27=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
            ##
            conv28=L.Convolution2D(out_chanel1,out_chanel1,ksize=k_size,pad=pad_size),
#             conv29=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv30=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             ##
#             conv31=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv32=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv33=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             ##
#             conv34=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv35=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv36=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             ##
#             conv37=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv38=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv39=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             ##
#             conv40=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv41=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv42=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             ##
#             conv43=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv44=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv45=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             ##
#             conv46=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv47=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv48=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             ##
#             conv49=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
#             conv50=L.Convolution2D(out_chanel1,out_chanel1,ksize=5,pad=2),
            
            
            bn0=L.BatchNormalization(out_chanel1),
            bn1=L.BatchNormalization(out_chanel1),
            bn2=L.BatchNormalization(out_chanel1),
            bn3=L.BatchNormalization(out_chanel1),
            bn4=L.BatchNormalization(out_chanel1),
            bn5=L.BatchNormalization(out_chanel1),
            bn6=L.BatchNormalization(out_chanel1),
            bn7=L.BatchNormalization(out_chanel1),
            bn8=L.BatchNormalization(out_chanel1),
            bn9=L.BatchNormalization(out_chanel1),
            bn10=L.BatchNormalization(out_chanel1),
            bn11=L.BatchNormalization(out_chanel1),
            bn12=L.BatchNormalization(out_chanel1),
            bn13=L.BatchNormalization(out_chanel1),
            bn14=L.BatchNormalization(out_chanel1),
            bn15=L.BatchNormalization(out_chanel1),
            bn16=L.BatchNormalization(out_chanel1),
            bn17=L.BatchNormalization(out_chanel1),
            bn18=L.BatchNormalization(out_chanel1),
            bn19=L.BatchNormalization(out_chanel1),
            bn20=L.BatchNormalization(out_chanel1),
            bn21=L.BatchNormalization(out_chanel1),
            bn22=L.BatchNormalization(out_chanel1),
            bn23=L.BatchNormalization(out_chanel1),
            bn24=L.BatchNormalization(out_chanel1),
            bn25=L.BatchNormalization(out_chanel1),
            bn26=L.BatchNormalization(out_chanel1),
            bn27=L.BatchNormalization(out_chanel1),
            bn28=L.BatchNormalization(out_chanel1),
#             bn29=L.BatchNormalization(out_chanel1),
#             bn30=L.BatchNormalization(out_chanel1),
#             bn31=L.BatchNormalization(out_chanel1),
#             bn32=L.BatchNormalization(out_chanel1),
#             bn33=L.BatchNormalization(out_chanel1),
#             bn34=L.BatchNormalization(out_chanel1),
#             bn35=L.BatchNormalization(out_chanel1),
#             bn36=L.BatchNormalization(out_chanel1),
#             bn37=L.BatchNormalization(out_chanel1),
#             bn38=L.BatchNormalization(out_chanel1),
#             bn39=L.BatchNormalization(out_chanel1),
#             bn40=L.BatchNormalization(out_chanel1),
#             bn41=L.BatchNormalization(out_chanel1),
#             bn42=L.BatchNormalization(out_chanel1),
#             bn43=L.BatchNormalization(out_chanel1),
#             bn44=L.BatchNormalization(out_chanel1),
#             bn45=L.BatchNormalization(out_chanel1),
#             bn46=L.BatchNormalization(out_chanel1),
#             bn47=L.BatchNormalization(out_chanel1),
#             bn48=L.BatchNormalization(out_chanel1),
#             bn49=L.BatchNormalization(out_chanel1),
#             bn50=L.BatchNormalization(out_chanel1),
            l1=L.Linear(None,2),
           
        
        )
        #self.conv_num = conv_num
        if activate == 'relu':
            self.act = F.relu
        elif activate == 'tanh':
            self.act = F.tanh
        elif activate == 'leakly_relu':
            self.act = F.leaky_relu
        
    def __call__(self,x):
            h0=self.act(self.bn0(self.conv0(x)))
            h1=self.act(self.bn1(self.conv1(h0)))
            h2=self.act(F.dropout(self.bn2(self.conv2(h1))))
            h3=self.act(self.bn3(self.conv3(h2))+h1)
            h4=self.act(F.dropout(self.bn4(self.conv4(h3))))
            h5=self.act(self.bn5(self.conv5(h4))+h3)
            h6=self.act(F.dropout(self.bn6(self.conv6(h5))))
            h7=self.act(self.bn7(self.conv7(h6))+h5)
            h8=self.act(F.dropout(self.bn8(self.conv8(h7))))
            h9=self.act(self.bn9(self.conv9(h8))+h7)
            h10=self.act(F.dropout(self.bn10(self.conv10(h9))))
            h11=self.act(self.bn11(self.conv11(h10))+h9)
            h12=self.act(F.dropout(self.bn12(self.conv12(h11))))
            h13=self.act(self.bn13(self.conv13(h12))+h11)
            h14=self.act(F.dropout(self.bn14(self.conv14(h13))))
            h15=self.act(self.bn15(self.conv15(h14))+h13)
            h16=self.act(F.dropout(self.bn16(self.conv16(h15))))
            h17=self.act(self.bn17(self.conv17(h16))+h15)
            h18=self.act(F.dropout(self.bn18(self.conv18(h17))))
            h19=self.act(self.bn19(self.conv19(h18))+h17)
            h20=self.act(F.dropout(self.bn20(self.conv20(h19))))
            h21=self.act(self.bn21(self.conv21(h20))+h19)
            h22=self.act(F.dropout(self.bn22(self.conv22(h21))))
            h23=self.act(self.bn23(self.conv23(h22))+h21)
            h24=self.act(F.dropout(self.bn24(self.conv24(h23))))
            h25=self.act(self.bn25(self.conv25(h24))+h23)
            h26=self.act(F.dropout(self.bn26(self.conv26(h25))))
            h27=self.act(self.bn27(self.conv27(h26))+h25)
            h28=self.act(F.dropout(self.bn28(self.conv28(h27))))
#             h29=self.act(self.bn29(self.conv29(h28))+h27)
#             h30=self.act(F.dropout(self.bn30(self.conv30(h29))))
#             h31=self.act(self.bn31(self.conv31(h30))+h29)
#             h32=self.act(F.dropout(self.bn32(self.conv32(h31))))
#             h33=self.act(self.bn33(self.conv33(h32))+h31)
#             h34=self.act(F.dropout(self.bn34(self.conv34(h33))))
#             h35=self.act(self.bn35(self.conv35(h34))+h33)
#             h36=self.act(F.dropout(self.bn36(self.conv36(h35))))
#             h37=self.act(self.bn37(self.conv37(h36))+h35)
#             h38=self.act(F.dropout(self.bn38(self.conv38(h37))))
#             h39=self.act(self.bn39(self.conv39(h38))+h37)
#             h40=self.act(F.dropout(self.bn40(self.conv40(h39))))
#             h41=self.act(self.bn41(self.conv41(h40))+h39)
#             h42=self.act(F.dropout(self.bn42(self.conv42(h41))))
#             h43=self.act(self.bn43(self.conv43(h42))+h41)
#             h44=self.act(F.dropout(self.bn44(self.conv44(h43))))
#             h45=self.act(self.bn45(self.conv45(h44))+h43)
#             h46=self.act(F.dropout(self.bn46(self.conv46(h45))))
#             h47=self.act(self.bn47(self.conv47(h46))+h45)
#             h48=self.act(F.dropout(self.bn48(self.conv48(h47))))
#             h49=self.act(self.bn49(self.conv49(h48))+h47)
#             h50=self.act(F.dropout(self.bn50(self.conv50(h49))))
            
            return self.l1(F.dropout(h28))
 

In [12]:
def main(params):
    epoch = 20
    gpu = 0
    batchsize = 256

    #out_chanel = params['out_chanel']
    #mid_units = params['mid_units']
    #conv_num = params['conv_num']
    activate = params['activate']
    optimizer_name = params['optimizer_name']
    lr = params['lr']
    #weight=params['weight']

    model = L.Classifier(Purchase( activate))
    
    if gpu >= 0:
        chainer.cuda.get_device(gpu).use()
        model.to_gpu()

    # Setup an optimizer
    if optimizer_name == 'Adam':
        optimizer = optimizers.Adam()
    elif optimizer_name == 'AdaDelta':
        optimizer = optimizers.AdaDelta()
    else:
        optimizer = optimizers.MomentumSGD(lr=lr)
    optimizer.setup(model)
    #optimizer.add_hook(chainer.optimizer_hooks.WeightDecay(weight))
    
    ##load data
    ##x,yのデータをtuple化して、更にvalidation dataを作成する

    data= tuple_dataset.TupleDataset(x,y)
    split_at=int(len(x)*0.95)
    train,test=chainer.datasets.split_dataset(data ,split_at)
    
    ##trainer
    train_iter = chainer.iterators.SerialIterator(train, batchsize)
    test_iter = chainer.iterators.SerialIterator(test, batchsize,repeat=False, shuffle=False)

    # Set up a trainer
    updater = training.StandardUpdater(train_iter, optimizer, device=gpu)
    trainer = training.Trainer(updater, (epoch, 'epoch'), out="hyperopt_result")
    
    # Evaluate the model with the test dataset for each epoch
    trainer.extend(extensions.Evaluator(test_iter, model, device=gpu))

    # Write a log of evaluation statistics for each epoch
    trainer.extend(extensions.LogReport())

    # Save two plot images to the result dir
    trainer.extend(
        extensions.PlotReport(['main/loss', 'validation/main/loss'],
                               'epoch', file_name='loss.png'))
    trainer.extend(
        extensions.PlotReport(
            ['main/accuracy', 'validation/main/accuracy'],
             'epoch', file_name='accuracy.png'))

    # Write report
    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

    # Print a progress bar to stdout
    trainer.extend(extensions.ProgressBar())

    # Run the training
    trainer.run()
    valid_data = trainer._extensions['PlotReport'].extension._data
    loss_data = [data for i, data in valid_data['validation/main/loss']]
    best10_loss = sorted(loss_data)[:10]
    return sum(best10_loss)

In [13]:
if __name__ == '__main__':
    ##探索空間の設定
    space = {
        #'out_chanel':scope.int(hp.quniform('out_chanel',32,256,32)),
        #'mid_units':scope.int(hp.quniform('mid_units',50,300,50)),
        #'conv_num':scope.int(hp.quniform('conv_num',2,20,1)),
        'activate':hp.choice('activate',('relu','tanh','leakly_relu')),
        'optimizer_name':hp.choice('optimizer_name',('Adam','AdaDelta','MomentumSGD')),
        'lr':hp.uniform('lr',0.005,0.02),
        #'weight':hp.uniform('weight',0.0001,0.005),
        
    }
    best = fmin(main, space, algo=tpe.suggest, max_evals=20)
    print("best parameters",best)

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
     total [..................................................]  1.05%
this epoch [##########........................................] 21.05%
       100 iter, 0 epoch / 20 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [#.................................................]  2.11%
this epoch [#####################.............................] 42.11%
       200 iter, 0 epoch / 20 epochs
   0.65931 iters/sec. Estimated time to finish: 3:55:05.708920.
     total [#.................................................]  3.16%
this epoch [###############################...................] 63.16%
       300 iter, 0 epoch / 20 epochs
   0.65993 iters/sec. Estimated time to finish: 3:52:20.838633.
     total [##................................................]  4.21%
this epoch [##########################################........] 84.21%
       400 iter, 0 epoch / 20 epochs

     total [################..................................] 32.63%
this epoch [##########################........................] 52.63%
      3100 iter, 6 epoch / 20 epochs
    0.6524 iters/sec. Estimated time to finish: 2:43:29.951334.
     total [################..................................] 33.68%
this epoch [####################################..............] 73.68%
      3200 iter, 6 epoch / 20 epochs
   0.65275 iters/sec. Estimated time to finish: 2:40:51.411674.
     total [#################.................................] 34.74%
this epoch [###############################################...] 94.74%
      3300 iter, 6 epoch / 20 epochs
    0.6531 iters/sec. Estimated time to finish: 2:38:13.233415.
7           0.688163    0.68806               0.548018       0.544531                  5102.7        
     total [#################.................................] 35.79%
this epoch [#######...........................................] 15.79%
      3400 iter, 7 epoch / 

13          0.675909    0.676355              0.582336       0.582656                  9466.39       
     total [################################..................] 65.26%
this epoch [##................................................]  5.26%
      6200 iter, 13 epoch / 20 epochs
   0.65234 iters/sec. Estimated time to finish: 1:24:18.700113.
     total [#################################.................] 66.32%
this epoch [#############.....................................] 26.32%
      6300 iter, 13 epoch / 20 epochs
   0.65254 iters/sec. Estimated time to finish: 1:21:43.937483.
     total [#################################.................] 67.37%
this epoch [#######################...........................] 47.37%
      6400 iter, 13 epoch / 20 epochs
   0.65272 iters/sec. Estimated time to finish: 1:19:09.329191.
     total [##################################................] 68.42%
this epoch [##################################................] 68.42%
      6500 iter, 13 epoc

     total [################################################..] 96.84%
this epoch [##################................................] 36.84%
      9200 iter, 19 epoch / 20 epochs
   0.65267 iters/sec. Estimated time to finish: 0:07:39.648497.
     total [################################################..] 97.89%
this epoch [############################......................] 57.89%
      9300 iter, 19 epoch / 20 epochs
   0.65277 iters/sec. Estimated time to finish: 0:05:06.384408.
     total [#################################################.] 98.95%
this epoch [#######################################...........] 78.95%
      9400 iter, 19 epoch / 20 epochs
   0.65287 iters/sec. Estimated time to finish: 0:02:33.169203.
20          0.670124    0.683057              0.590107       0.576875                  14561.2       
     total [##################################################] 100.00%
this epoch [..................................................]  0.00%
      9500 iter, 20 epo

     total [##############....................................] 28.42%
this epoch [##################################................] 68.42%
      2700 iter, 5 epoch / 20 epochs
   0.65181 iters/sec. Estimated time to finish: 2:53:52.502172.
     total [##############....................................] 29.47%
this epoch [############################################......] 89.47%
      2800 iter, 5 epoch / 20 epochs
   0.65218 iters/sec. Estimated time to finish: 2:51:13.208804.
6           0.81062     0.739748              0.57287        0.579219                  4378.5        
     total [###############...................................] 30.53%
this epoch [#####.............................................] 10.53%
      2900 iter, 6 epoch / 20 epochs
   0.65065 iters/sec. Estimated time to finish: 2:49:03.761466.
     total [###############...................................] 31.58%
this epoch [###############...................................] 31.58%
      3000 iter, 6 epoch / 

     total [##############################....................] 60.00%
this epoch [..................................................]  0.00%
      5700 iter, 12 epoch / 20 epochs
   0.65046 iters/sec. Estimated time to finish: 1:37:22.009339.
     total [##############################....................] 61.05%
this epoch [##########........................................] 21.05%
      5800 iter, 12 epoch / 20 epochs
   0.65083 iters/sec. Estimated time to finish: 1:34:45.076504.
     total [###############################...................] 62.11%
this epoch [#####################.............................] 42.11%
      5900 iter, 12 epoch / 20 epochs
   0.65102 iters/sec. Estimated time to finish: 1:32:09.800837.
     total [###############################...................] 63.16%
this epoch [###############################...................] 63.16%
      6000 iter, 12 epoch / 20 epochs
    0.6512 iters/sec. Estimated time to finish: 1:29:34.667691.
     total [############

     total [##############################################....] 92.63%
this epoch [##########################........................] 52.63%
      8800 iter, 18 epoch / 20 epochs
   0.65107 iters/sec. Estimated time to finish: 0:17:55.149771.
     total [##############################################....] 93.68%
this epoch [####################################..............] 73.68%
      8900 iter, 18 epoch / 20 epochs
    0.6512 iters/sec. Estimated time to finish: 0:15:21.382655.
     total [###############################################...] 94.74%
this epoch [###############################################...] 94.74%
      9000 iter, 18 epoch / 20 epochs
   0.65132 iters/sec. Estimated time to finish: 0:12:47.675510.
19          0.720634    0.766672              0.591579       0.587969                  13865.4       
     total [###############################################...] 95.79%
this epoch [#######...........................................] 15.79%
      9100 iter, 19 epoc

     total [############......................................] 24.21%
this epoch [##########################################........] 84.21%
      2300 iter, 4 epoch / 20 epochs
   0.65235 iters/sec. Estimated time to finish: 3:03:57.082172.
5           0.759101    0.704942              0.513067       0.492656                  3648.74       
     total [############......................................] 25.26%
this epoch [##................................................]  5.26%
      2400 iter, 5 epoch / 20 epochs
    0.6505 iters/sec. Estimated time to finish: 3:01:54.761382.
     total [#############.....................................] 26.32%
this epoch [#############.....................................] 26.32%
      2500 iter, 5 epoch / 20 epochs
   0.65097 iters/sec. Estimated time to finish: 2:59:13.116686.
     total [#############.....................................] 27.37%
this epoch [#######################...........................] 47.37%
      2600 iter, 5 epoch / 

KeyboardInterrupt: 